In [1]:
!pip install fsspec==2025.3.2
!pip install datasets transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset

# Example
ds = load_dataset("tau/commonsense_qa", split="train[:5]")
print(ds)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 5
})


In [5]:
from datasets import load_dataset

ds          = load_dataset("tau/commonsense_qa", split="train[:5]")
questions   = ds["question"]
choices_all = ds["choices"]
answer_keys = ds["answerKey"]   # list of "A", "B", ...

texts = []
for q, choice_dict in zip(questions, choices_all):
    prompt = "Question: " + q + "\n" + "\n".join(
        f"{lbl}. {txt}"
        for lbl, txt in zip(choice_dict["label"], choice_dict["text"])
    )
    texts.append(prompt)

labels = []
for choice_dict, key in zip(choices_all, answer_keys):
    # find the index of the correct label letter in the 'label' list
    idx = choice_dict["label"].index(key)
    # grab that text
    labels.append(choice_dict["text"][idx])

print(texts[0])
print("→ label:", labels[0])

Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
A. ignore
B. enforce
C. authoritarian
D. yell at
E. avoid
→ label: ignore


In [2]:
# Install prerequisites
!pip install transformers datasets sentence-transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d672586d1406046038581d1edcbb0eb01e3a8f40208d3cf5feb08accb77b0b03
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


## Commonsense qa

In [2]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

class Se2ICLInference:
    def __init__(
        self,
        model_name: str = "EleutherAI/gpt-neo-1.3B",   # ← larger model
        embed_model: str = "all-MiniLM-L6-v2",
        shot: int = 3,
        retrieve_k: int = 20,      # fewer candidates for speed
        beam_size: int = 3,
        device: str = None,
    ):
        self.device     = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 1) Bi-encoder for fast retrieval
        self.embedder   = SentenceTransformer(embed_model, device=self.device)

        # 2) Causal LM for CE and generation
        self.tok        = AutoTokenizer.from_pretrained(model_name)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm         = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    def _batch_ce(self, texts):
        enc = self.tok(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(self.device)
        with torch.no_grad():
            out = self.lm(enc.input_ids, labels=enc.input_ids)
        lengths = enc.attention_mask.sum(dim=1).float()
        return (out.loss * lengths).cpu().tolist()

    def beam_search(self, prompt: str, label: str):
        # pre-filter top-K by cosine
        q_emb      = self.embedder.encode([prompt], convert_to_tensor=True)
        sims, idxs = (self.demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        cands      = [self.demos[i] for i in idxs.cpu().tolist()]

        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, batch_xc, batch_c, parents = [], [], [], []
            for seq, score in beams:
                for d in cands:
                    if d in seq:
                        continue
                    batch_xc.append(f"{d}\n{prompt}\nAnswer: {label}")
                    batch_c .append(f"{d}\nAnswer: {label}")
                    parents.append((seq, d, score))
            H_xc = self._batch_ce(batch_xc)
            H_c  = self._batch_ce(batch_c)
            for (seq, d, score), hxc, hc in zip(parents, H_xc, H_c):
                new_beams.append((seq+[d], score + (hxc-hc)))
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[: self.beam_size]
        return beams[0][0]

    def generate(self, prompt: str, demos: list[str]) -> str:
        full = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc  = self.tok(full, return_tensors="pt").to(self.device)
        gen  = self.lm.generate(
            **enc, max_new_tokens=10, pad_token_id=self.tok.eos_token_id
        )
        out  = self.tok.decode(gen[0], skip_special_tokens=True)
        return out.rsplit("Answer:",1)[-1].strip().split()[0]

    def evaluate(self, prompts, labels, demos):
        # precompute once
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, l) in enumerate(zip(prompts, labels)):
            demos_sel = self.beam_search(p, l)
            pred      = self.generate(p, demos_sel)

            ok = (pred.lower() == l.lower())
            correct += ok

            # print the first few examples in detail
            if i < 5:
                print(f"\n=== Example {i} ===")
                print(p)
                print("Selected demos:")
                for d in demos_sel:
                    print("  -", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r},  Gold: {l!r} →", "✓" if ok else "✗")

        acc = correct / len(prompts)
        print(f"\n⇒ Overall Accuracy: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__=="__main__":
    # 1) load a small slice
    ds = load_dataset("tau/commonsense_qa", split="train[:100]")

    # 2) build prompts & labels
    prompts, labels = [], []
    for ex in ds:
        q        = ex["question"]
        labs,txt = ex["choices"]["label"], ex["choices"]["text"]
        p        = "Question: " + q + "\n" + "\n".join(f"{L}. {T}"
                                                        for L, T in zip(labs, txt))
        prompts.append(p)
        labels.append(txt[labs.index(ex["answerKey"])])

    # 3) build demo‐pool
    demos = [f"{p}\nAnswer: {l}" for p, l in zip(prompts, labels)]

    # 4) run Se₂
    selector = Se2ICLInference(
        model_name   = "EleutherAI/gpt-neo-1.3B",
        embed_model  = "all-MiniLM-L6-v2",
        shot         = 3,
        retrieve_k   = 20,
        beam_size    = 3
    )
    selector.evaluate(prompts, labels, demos)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]


=== Example 0 ===
Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
A. ignore
B. enforce
C. authoritarian
D. yell at
E. avoid
Selected demos:
  - Question: Who was the head of the branch yelling at? ↵ A. owner ↵ B. foot ↵ C. subordinate ↵ D. base ↵ E. point ↵ Answer: subordinate
  - Question: The president is the leader of what institution? ↵ A. walmart ↵ B. white house ↵ C. country ↵ D. corporation ↵ E. government ↵ Answer: government
  - Question: The evacuation became mandatory, so what came on the TV? ↵ A. advisory ↵ B. mandate ↵ C. soap opera ↵ D. elective ↵ E. optional ↵ Answer: advisory
Predicted: 'ignore',  Gold: 'ignore' → ✓

=== Example 1 ===
Question: Sammy wanted to go to where the people were.  Where might he go?
A. race track
B. populated areas
C. the desert
D. apartment
E. roadblock
Selected demos:
  - Question: A person lived in my what? ↵ A. demonstration ↵ B. conference ↵ C. building ↵

## SST 5

In [2]:
# ── Install once: ────────────────────────────────────────────────────────────
# pip install transformers datasets sentence-transformers

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

class Se2ICLInference:
    def __init__(
        self,
        model_name: str = "EleutherAI/gpt-neo-1.3B",
        embed_model: str = "all-MiniLM-L6-v2",
        shot: int = 3,
        retrieve_k: int = 20,
        beam_size: int = 3,
        device: str = None,
    ):
        self.device     = device or ("cuda" if torch.cuda.is_available() else "cpu")
        # 1) SBERT for fast retrieval
        self.embedder   = SentenceTransformer(embed_model, device=self.device)
        # 2) GPT-Neo for CE scoring & generation
        self.tok        = AutoTokenizer.from_pretrained(model_name)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm         = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    def _batch_ce(self, texts):
        """Compute total cross-entropy for a batch of strings."""
        enc = self.tok(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(self.device)
        with torch.no_grad():
            out = self.lm(enc.input_ids, labels=enc.input_ids)
        lengths = enc.attention_mask.sum(dim=1).float()
        return (out.loss * lengths).cpu().tolist()

    def beam_search(self, prompt: str, label: str):
        # 1) retrieve top-K demos by SBERT cosine
        q_emb      = self.embedder.encode([prompt], convert_to_tensor=True)
        sims, idxs = (self.demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [self.demos[i] for i in idxs.cpu().tolist()]

        # 2) beam-search with CE re-ranking
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, batch_xc, batch_c, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    # score with and without the query
                    batch_xc.append(f"{d}\nReview: {prompt}\nSentiment: {label}")
                    batch_c .append(f"{d}\nSentiment: {label}")
                    parents.append((seq, d, cum))
            H_xc = self._batch_ce(batch_xc)
            H_c  = self._batch_ce(batch_c)
            for (seq, d, cum), hxc, hc in zip(parents, H_xc, H_c):
                new_beams.append((seq + [d], cum + (hxc - hc)))
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[: self.beam_size]
        return beams[0][0]

    def generate(self, prompt: str, demos: list[str]) -> str:
        full = "\n\n".join(demos) + f"\n\nReview: {prompt}\nSentiment:"
        enc  = self.tok(full, return_tensors="pt").to(self.device)
        gen  = self.lm.generate(
            **enc,
            max_new_tokens=5,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        # first token after the final "Sentiment:"
        return out.rsplit("Sentiment:", 1)[-1].strip().split()[0]

    def evaluate(self, prompts, labels, demos):
        # precompute once
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, l) in enumerate(zip(prompts, labels)):
            sel  = self.beam_search(p, l)
            pred = self.generate(p, sel)
            ok   = (pred.lower() == l.lower())
            correct += ok

            if i < 3:  # show first 3 cases
                print(f"\n=== Example {i} ===")
                print("Review:", p)
                print("Selected demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {l!r} →", "✓" if ok else "✗")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__ == "__main__":
    # 1) load a small slice of SST-5 for speed
    ds = load_dataset("SetFit/sst5", split="train[:200]")

    # 2) build prompts and string-labels
    label_names = [
        "Very negative", "Negative", "Neutral", "Positive", "Very positive"
    ]
    prompts, labels = [], []
    for ex in ds:
        text = ex["text"]
        prompts.append(text)
        labels.append(label_names[ex["label"]])

    # 3) demo-pool entries: each is "<review>\nSentiment: <label>"
    demos = [f"{r}\nSentiment: {l}" for r, l in zip(prompts, labels)]

    # 4) run Se₂
    selector = Se2ICLInference(
        model_name   = "EleutherAI/gpt-neo-1.3B",
        embed_model  = "all-MiniLM-L6-v2",
        shot         = 3,
        retrieve_k   = 20,
        beam_size    = 3,
    )
    selector.evaluate(prompts, labels, demos)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/171k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]


=== Example 0 ===
Review: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films
Selected demos:
 • i could n't recommend this film more . ↵ Sentiment: Positive
 • this is a gorgeous film - vivid with color , music and life . ↵ Sentiment: Very positive
 • a perfectly acceptable , perfectly bland , competently acted but by no means scary horror movie . ↵ Sentiment: Neutral
Predicted: 'Positive', Gold: 'Very positive' → ✗

=== Example 1 ===
Review: apparently reassembled from the cutting-room floor of any given daytime soap .
Selected demos:
 • the movie is so thoughtlessly assembled . ↵ Sentiment: Positive
 • a metaphor for a modern-day urban china searching for its identity . ↵ Sentiment: Positive
 • apparently reassembled from the cutting-room floor of any given daytime soap . ↵ Sentiment: Negative
Predicted: 'Negative', Gold: 'Negative' → ✓

=== Example 2 ===
Review: they presume their audience wo n't sit still for a sociology lesson 

## AG News

In [3]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers

import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

class Se2ICLInference:
    def __init__(
        self,
        model_name: str = "EleutherAI/gpt-neo-1.3B",
        embed_model: str = "all-MiniLM-L6-v2",
        shot: int = 3,
        retrieve_k: int = 20,
        beam_size: int = 3,
        device: str = None,
    ):
        # select device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 1) Bi-encoder for fast retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 2) Causal LM for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.lm.eval()

        # hyperparams
        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    def _batch_ce(self, texts):
        """Compute total cross‐entropy for each input in `texts`."""
        enc = self.tok(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(self.device)
        with torch.no_grad():
            out = self.lm(enc.input_ids, labels=enc.input_ids)
        lengths = enc.attention_mask.sum(dim=1).float()
        return (out.loss * lengths).cpu().tolist()

    def beam_search(self, prompt: str, label: str):
        """Pick `shot` demos via CE beam‐search over top‐K SBERT retrievals."""
        # retrieve top‐K candidates
        q_emb      = self.embedder.encode([prompt], convert_to_tensor=True)
        sims, idxs = (self.demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [self.demos[i] for i in idxs.cpu().tolist()]

        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, batch_xc, batch_c, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    # CE with and without the article
                    batch_xc.append(f"{d}\n\nArticle:\n{prompt}\nCategory: {label}")
                    batch_c .append(f"{d}\n\nCategory: {label}")
                    parents.append((seq, d, cum))
            H_xc = self._batch_ce(batch_xc)
            H_c  = self._batch_ce(batch_c)
            for (seq, d, cum), hxc, hc in zip(parents, H_xc, H_c):
                new_beams.append((seq + [d], cum + (hxc - hc)))
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[: self.beam_size]

        return beams[0][0]

    def generate(self, prompt: str, demos: list[str]) -> str:
        """Generate a one‐token category after 'Category:'."""
        full = "\n\n".join(demos) + f"\n\nArticle:\n{prompt}\nCategory:"
        enc  = self.tok(full, return_tensors="pt").to(self.device)
        gen  = self.lm.generate(
            **enc,
            max_new_tokens=5,
            pad_token_id=self.tok.eos_token_id
        )
        out  = self.tok.decode(gen[0], skip_special_tokens=True)
        # first token after final "Category:"
        return out.rsplit("Category:", 1)[-1].strip().split()[0]

    def evaluate(self, prompts, labels, demos):
        # precompute SBERT embeddings for the demo‐pool once
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, l) in enumerate(zip(prompts, labels)):
            sel  = self.beam_search(p, l)
            pred = self.generate(p, sel)
            ok   = (pred.lower() == l.lower())
            correct += ok

            # show first 3 examples for inspection
            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article:", p[:200].replace("\n"," "))
                print("Selected demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {l!r} →", "✓" if ok else "✗")

        acc = correct / len(prompts)
        print(f"\n⇒ AG News Accuracy: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__=="__main__":
    # 1) load a small slice of AG News (for speed)
    ds = load_dataset("ag_news", split="train[:200]").shuffle(seed=0)

    # 2) map numeric labels to human‐readable text
    label_map = ["World", "Sports", "Business", "Sci/Tech"]
    prompts, labels = [], []
    for ex in ds:
        text = ex["text"]
        prompts.append(text)
        labels.append(label_map[ex["label"]])

    # 3) demo‐pool: each "<article>\nCategory: <label>"
    demos = [f"{t}\nCategory: {l}" for t, l in zip(prompts, labels)]

    # 4) instantiate & evaluate
    selector = Se2ICLInference(
        model_name   = "EleutherAI/gpt-neo-1.3B",
        embed_model  = "all-MiniLM-L6-v2",
        shot         = 3,
        retrieve_k   = 20,
        beam_size    = 3,
    )
    selector.evaluate(prompts, labels, demos)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]


=== Example 0 ===
Article: Armadillo Aerospaces X Prize Prototype Crashes (SPACE.com) SPACE.com - Armadillo Aerospace of Mesquite, Texas has reported a \crash last weekend of their prototype X Prize rocket.
Selected demos:
 • GameBoy mini-games win prize A set of GameBoy micro-games is named as the most innovative game of the year at a festival in Scotland. ↵ Category: Sci/Tech
 • Canadian Robot a Candidate to Save Hubble (AP) AP - NASA said Tuesday it is moving ahead with plans to send a robot to the rescue of the aging Hubble Space Telescope. ↵ Category: Sci/Tech
 • Armadillo Aerospaces X Prize Prototype Crashes (SPACE.com) SPACE.com - Armadillo Aerospace of Mesquite, Texas has reported a \crash last weekend of their prototype X Prize rocket. ↵ Category: Sci/Tech
Predicted: 'Sci/Tech', Gold: 'Sci/Tech' → ✓

=== Example 1 ===
Article: Money Funds Fell in Latest Week (AP) AP - Assets of the nation's retail money market mutual funds fell by  #36;1.17 billion in the latest week to  #36;